In [22]:
import numpy as np
import pandas as pd

from jupyter_datatables import init_datatables_mode

import optuna

import catboost as cb
from sklearn import model_selection, metrics, linear_model, ensemble, naive_bayes, neighbors, svm, tree
from xgboost import XGBClassifier
import lightgbm

In [24]:
#init_datatables_mode()

In [25]:
train = pd.read_csv(r'./data/train.csv')
train_o = pd.read_csv(r'./data/creditcard.csv')
test = pd.read_csv(r'./data/test.csv')
ss = pd.read_csv(r'./data/sample_submission.csv')

In [26]:
print(train.shape, train_o.shape, test.shape)

(219129, 32) (284807, 31) (146087, 31)


In [27]:
train.isnull().sum() + train_o.isnull().sum()

Amount    0.0
Class     0.0
Time      0.0
V1        0.0
V10       0.0
V11       0.0
V12       0.0
V13       0.0
V14       0.0
V15       0.0
V16       0.0
V17       0.0
V18       0.0
V19       0.0
V2        0.0
V20       0.0
V21       0.0
V22       0.0
V23       0.0
V24       0.0
V25       0.0
V26       0.0
V27       0.0
V28       0.0
V3        0.0
V4        0.0
V5        0.0
V6        0.0
V7        0.0
V8        0.0
V9        0.0
id        NaN
dtype: float64

In [28]:
display(train.head())
display(train_o.head())

id  Time        V1        V2        V3        V4        V5        V6  \
0   0   0.0  2.074329 -0.129425 -1.137418  0.412846 -0.192638 -1.210144   
1   1   0.0  1.998827 -1.250891 -0.520969 -0.894539 -1.122528 -0.270866   
2   2   0.0  0.091535  1.004517 -0.223445 -0.435249  0.667548 -0.988351   
3   3   0.0  1.979649 -0.184949 -1.064206  0.120125 -0.215238 -0.648829   
4   4   0.0  1.025898 -0.171827  1.203717  1.243900 -0.636572  1.099074   

         V7        V8  ...       V21       V22       V23       V24       V25  \
0  0.110697 -0.263477  ... -0.334701 -0.887840  0.336701 -0.110835 -0.291459   
1 -1.029289  0.050198  ...  0.054848 -0.038367  0.133518 -0.461928 -0.465491   
2  0.948146 -0.084789  ... -0.326725 -0.803736  0.154495  0.951233 -0.506919   
3 -0.087826 -0.035367  ... -0.095514 -0.079792  0.167701 -0.042939  0.000799   
4 -0.938651  0.569239  ...  0.099157  0.608908  0.027901 -0.262813  0.257834   

        V26       V27       V28  Amount  Class  
0  0.207733 -0.076576 -0.059577    1.98      0  
1 -0.464655 -0.009413 -0.038238   84.00      0  
2  0.085046  0.224458  0.087356    2.69      0  
3 -0.096148 -0.057780 -0.073839    1.00      0  
4 -0.252829  0.108338  0.021051    1.00      0  

[5 rows x 32 columns]

Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

In [29]:
display(train['Class'].value_counts())
display(train_o['Class'].value_counts())

0    218660
1       469
Name: Class, dtype: int64

0    284315
1       492
Name: Class, dtype: int64

In [31]:
train_all = pd.concat([train.drop('id', axis = 1), train_o])

In [32]:
train_all

Time         V1         V2        V3        V4        V5  \
0            0.0   2.074329  -0.129425 -1.137418  0.412846 -0.192638   
1            0.0   1.998827  -1.250891 -0.520969 -0.894539 -1.122528   
2            0.0   0.091535   1.004517 -0.223445 -0.435249  0.667548   
3            0.0   1.979649  -0.184949 -1.064206  0.120125 -0.215238   
4            0.0   1.025898  -0.171827  1.203717  1.243900 -0.636572   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0      -1.210144  0.110697 -0.263477  0.742144  ... -0.334701 -0.887840   
1      -0.270866 -1.029289  0.050198 -0.109948  ...  0.054848 -0.038367   
2      -0.988351  0.948146 -0.084789 -0.042027  ... -0.326725 -0.803736   
3      -0.648829 -0.087826 -0.035367  0.885838  ... -0.095514 -0.079792   
4       1.099074 -0.938651  0.569239  0.692665  ...  0.099157  0.608908   
...          ...       ...       ...       ...  ...       ...       ...   
284802 -2.606837 -4.918215  7.305334  1.914428  ...  0.213454  0.111864   
284803  1.058415  0.024330  0.294869  0.584800  ...  0.214205  0.924384   
284804  3.031260 -0.296827  0.708417  0.432454  ...  0.232045  0.578229   
284805  0.623708 -0.686180  0.679145  0.392087  ...  0.265245  0.800049   
284806 -0.649617  1.577006 -0.414650  0.486180  ...  0.261057  0.643078   

             V23       V24       V25       V26       V27       V28  Amount  \
0       0.336701 -0.110835 -0.291459  0.207733 -0.076576 -0.059577    1.98   
1       0.133518 -0.461928 -0.465491 -0.464655 -0.009413 -0.038238   84.00   
2       0.154495  0.951233 -0.506919  0.085046  0.224458  0.087356    2.69   
3       0.167701 -0.042939  0.000799 -0.096148 -0.057780 -0.073839    1.00   
4       0.027901 -0.262813  0.257834 -0.252829  0.108338  0.021051    1.00   
...          ...       ...       ...       ...       ...       ...     ...   
284802  1.014480 -0.509348  1.436807  0.250034  0.943651  0.823731    0.77   
284803  0.012463 -1.016226 -0.606624 -0.395255  0.068472 -0.053527   24.79   
284804 -0.037501  0.640134  0.265745 -0.087371  0.004455 -0.026561   67.88   
284805 -0.163298  0.123205 -0.569159  0.546668  0.108821  0.104533   10.00   
284806  0.376777  0.008797 -0.473649 -0.818267 -0.002415  0.013649  217.00   

        Class  
0           0  
1           0  
2           0  
3           0  
4           0  
...       ...  
284802      0  
284803      0  
284804      0  
284805      0  
284806      0  

[503936 rows x 31 columns]

In [33]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_all.drop('Class', axis = 1), train_all.Class, random_state=42)

In [34]:
xgb = XGBClassifier()

xgb.fit(X_train, y_train)

metrics.roc_auc_score(y_test, xgb.predict(X_test))

0.7111210030129503

array([[9.9871665e-01, 1.2833365e-03],
       [9.9999058e-01, 9.4442048e-06],
       [9.9999148e-01, 8.5042402e-06],
       ...,
       [9.9999154e-01, 8.4690737e-06],
       [9.9999410e-01, 5.9241206e-06],
       [9.9999607e-01, 3.9629817e-06]], dtype=float32)

In [39]:
ss[['Class_0','Class']] = xgb.predict_proba(test.drop('id', axis = 1))

In [41]:
ss[['id', 'Class']].to_csv('submission.csv', index = False)